In [1]:
import glob
import pandas as pd
import requests
import json

In [2]:
pd.set_option('display.max_columns', 23)

In [3]:
path = "../../vocabulary/*.csv"
examples = pd.DataFrame()

for file in glob.glob(path):
    approach = pd.read_csv(file)
    examples = examples.append(approach, ignore_index=True)

In [4]:
def lookup_metadata(field):
    line = examples[examples['field_id'] == field]
    line = line.dropna(how='all', axis=1)
    return line

In [5]:
lookup_metadata(2986)

,field_id,name,mapping_approach,date_field_id,modified_by_field_id,modified_by_value,target_concept_id,target_value_concept_id,value,value_name
4,2986,Started insulin within one year diagnosis of d...,Modified,53.0,31/2443/4041,1/1,4188893.0,45884660.0,1.0,Yes
5,2986,Started insulin within one year diagnosis of d...,Modified,53.0,31/2443/4041,0/1/0,4188893.0,45884660.0,0.0,No


In [6]:
def omop_domain(field):
    concept_id = int(examples[examples['field_id'] == field]['target_concept_id'].unique())
    response = requests.get(f'https://athena.ohdsi.org/api/v1/concepts/{concept_id}')
    concept = json.loads(response.text)
    domain = concept['domainId']
    return domain

In [7]:
def example_omop_table(field, instance, *value):
    domain = omop_domain(field)
    if value:
        row = examples[(examples['field_id'] == field) & (examples['value'] == value)]
    else:
        row = examples[examples['field_id'] == field]

    table = pd.Series({'person_id': 'get(eid)',
                       domain+'_concept_id': str(row.iloc[0]['target_concept_id']),
                       domain+'_date': 'get date from field '+str(row.iloc[0]['date_field_id'])+'-'+instance,
                       'value_as_concept_id': str(row.iloc[0]['target_value_concept_id']),
                       'value_as_number': 'get value from field '+str(row.iloc[0]['field_id'])+'-'+instance,
                       'unit_concept_id': str(row.iloc[0]['unit_concept_id'])
                       })
    return table

In [8]:
omop_domain(2986)

'Observation'

In [9]:
example_omop_table(2986, '0.1', 0)


person_id                                      get(eid)
Observation_concept_id                        4188893.0
Observation_date           get date from field 53.0-0.1
value_as_concept_id                          45884660.0
value_as_number           get value from field 2986-0.1
unit_concept_id                                     nan
dtype: object